# Lecture 10: linear model and model diagnostics 

## Instructor： 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]

##  Recap: Linear Model and model diagnositcs

* Workflow
* MCMC diagnostics

我们在上节课中采用一个简化的workflow，具体包括如下几个步骤：



![Image Name](https://cdn.kesci.com/upload/image/rkvikqg9q6.png?imageView2/0/w/960/h/960)


现在，我们通过上节课的例子来简单回顾一下workflow

在本例中，涉及：研究问题、数据收集、选择模型、选择先验、模型拟合、采样过程评估、模型诊断

### (1) 提出研究问题

还有研究发现个体创新行为可能与自尊水平有关。
SES_t:自尊水平；
EIB_t创新行为

试探究两者之间的关系。

### (2) 数据收集

In [15]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm
# mpl_toolkits.mplot3d是用于三维画图的，Axes3D用于画三维图
from mpl_toolkits.mplot3d import Axes3D
#数据预处理与可视化
np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同
data = pd.read_csv("/home/mw/input/data9464/clean.csv") # 读取数据，该数据需要提前挂载
data['SES_t'] = (data['SES_t'] - data['SES_t'].mean()) / data['SES_t'].std()#将变量进行标准化

data['EIB_t'] = (data['EIB_t'] - data['EIB_t'].mean()) / data['EIB_t'].std()#将变量进行标准化

plt.scatter(data['SES_t'],data['EIB_t'])
plt.xlabel('SES_t')
plt.ylabel('EIB_t')

Text(0, 0.5, 'EIB_t')

<Figure size 432x288 with 1 Axes>

### (3) 选择模型

尝试构建一个简单的线性模型

线性模型可以用概率的形式进行表达


$\alpha \sim Normal(0,1)$      ->      a $\sim$ Normal(mu,sigma)

$\beta  \sim Normal(0,1)$      ->      b $\sim$ Normal(mu,sigma)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)

$\mu_i  = \alpha + \beta *x$      ->      mu = alpha + beta*x 

$y \sim Normal(\mu_i,sigma)$      ->      y $\sim$ Normal(mu,sigma)

### (4)选择先验

In [16]:
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1, shape=1)  
    sigma = pm.HalfNormal('sigma',sd=1)
    
    # 先验预测检查
    prior_checks = pm.sample_prior_predictive(samples=50)

In [17]:
fig, ax = plt.subplots()
x1 = np.linspace(-3, 3, 50) #生成从-2，2之间的50个假数据

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y = a + b * x1 # 基于假数据生成预测值
    ax.plot(x1,y)

<Figure size 432x288 with 1 Axes>

### (5) 拟合数据

In [18]:
linear_model = pm.Model()
with linear_model :
    # 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
    # 在python中，容器是一种数据结构，是用来管理特殊数据的对象
    # with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
    alpha = pm.Normal('alpha',mu=0,sd=1)
    beta = pm.Normal('beta',mu=0,sd=1,shape=1)
    sigma = pm.HalfNormal('sigma',sd=1)
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data['SES_t'])
    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=data['EIB_t'])

### (6)采样过程诊断

如果使用MCMC对后验进行近似，则需要首先对MCMC过程进行评估。

* 是否收敛；
* 是否接近真实的后验。

对采样过程的评估我们会采用目视检查或rhat这个指标

In [19]:
with linear_model :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


In [20]:
az.plot_trace(trace,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [21]:
az.summary(trace, var_names=['alpha','beta','sigma'], kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.0,0.0,5018.0,2745.0,1.0
beta[0],0.0,0.0,4503.0,3309.0,1.0
sigma,0.0,0.0,4367.0,3085.0,1.0


### (7)模型诊断

在MCMC有效的前提下，需要继续检验模型是否能够较好地拟合数据。

我们会使用后验预测分布通过我们得到的参数生成一批模拟数据，并将其与真实数据进行对比。

In [22]:
# 后验预测分布的计算仍在容器中进行
with linear_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [23]:
# 绘制后验预测分布
az.plot_ppc(trace)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/events.py:89: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args, **kwargs)


<Figure size 432x288 with 1 Axes>

## Part 2: Model comparation

我们在上节课中提到的workflow具体包括如下几个步骤：

研究问题、选择模型、选择先验、模型拟合、采样过程评估、模型诊断、模型比较、统计推断、结果报告。

![Image Name](https://cdn.kesci.com/upload/image/rkm3pw954u.png?imageView2/0/w/960/h/960)


在贝叶斯的workflow中，模型诊断就是对MCMC进行检验，检验的核心在于确定MCMC算法采样得到的样本是否足以提供目标分布的精确近似。

模型诊断结果表明MCMC算法采样得到的样本能够提供目标分布的精确近似之后，我们需要确定模型既不太简单以至于错过数据中有价值的信息(underfitting)，也不会太复杂从而过拟合数据中的噪音(overfitting)。


![Image Name](https://img-blog.csdnimg.cn/20210303141100499.png#pic_center)

资料来源：https://blog.csdn.net/weixin_43378396/article/details/90707493

### 欠拟合(underfitting)

欠拟合的模型在当前样本的数据拟合效果不好，且其泛化能力(模型在当前样本外新的数据上的预测的准确度)也同样不佳。

### 过拟合(overfitting)

过拟合的模型在当前样本的数据上的拟合程度很好，但是泛化能力也较差。

这是因为模型把训练样本学习地“太好了”，把样本自身地一些特点也当作了所有潜在样本都会具有的一些性质，这样就会导致其泛化性能下降。


### 为什么会发生欠拟合

- 数据特征较少

数据特征指的是数据的属性，比如part1中展示的数据的各个变量就是数据的特征。在所有变量都能独立地对目标变量做出解释的前提下，数据特征越多，数据拟合程度越好。

- 模型复杂度过低

模型的复杂度代表模型能够描述的所有函数，比如线性回归最多能表示所有的线性函数。

模型的复杂度和模型的参数数量有关，一般来说，模型参数越多，复杂度越高，模型参数越少，复杂度越低。

### 为什么会发生过拟合

- 当前样本的噪音过大，模型将噪音当作数据本身的特征

当数据的有些特征与目标变量无关，这些特征就是噪音，但它也可能被误当作数据特征，这就会造成模型过拟合

- 样本选取有误，样本不能代表整体

- 模型参数太多，模型复杂度太高



![Image Name](https://vitalflux.com/wp-content/uploads/2020/12/overfitting-and-underfitting-wrt-model-error-vs-complexity-768x443.png)

资料来源：https://vitalflux.com/overfitting-underfitting-concepts-interview-questions/

### 如何避免欠拟合

- 增加数据的特征

- 增加模型复杂度

### 如何避免过拟合

- 选择更具代表性的数据

- 降低模型复杂度

为了选择一个能够在过拟合和欠拟合之间的达到平衡的最佳模型，就需要进行模型比较。

根据不同模型比较的指标的关注点和原理，进行模型比较的统计指标可被分为三类。 

第一类是模型的拟合优度(Goodness of fit)

第二类是交叉验证(Cross validation)和近似交叉验证

第三类是边缘似然理论的指标



### 拟合优度(Goodness of fit)

拟合优度衡量模型在样本集上拟合的程度，并没有考虑模型的复杂度，只是单纯地用于衡量模型在当前样本的拟合的好坏。

比如回归模型中的$R^2$，即回归模型所能解释的因变量的变异的百分比。

常用的指标有均方差(Mean squared deviation, MSD)，对数似然((log likelihood function)



![Image Name](https://th.bing.com/th/id/R.f8319827592edfae31cd1248ebb7be01?rik=UksWyJLwHYJqXQ&pid=ImgRaw&r=0)



资料来源：https://baike.baidu.com/pic/%E6%8B%9F%E5%90%88%E4%BC%98%E5%BA%A6/10946257/0/6a63f6246b600c338b38d998114c510fd9f9a118

#### 均方差

计算公式为：$MSE(\hat{\theta})=E(\hat{y}-y)^2=\frac{1}{n}\sum (\hat{y}-y)^2$

均方差表示预测变量与目标变量之间差值的平方之和的均值


![Image Name](https://cdn-media-1.freecodecamp.org/images/MNskFmGPKuQfMLdmpkT-X7-8w2cJXulP3683)

资料来源：https://www.freecodecamp.org/news/machine-learning-mean-squared-error-regression-line-c7dde9a26b93/



MSE与许多常见的拟合优度指标有关联，

对MSE开根号，可以得到**均方根误差(Root mean square deviation, RMSD)**;

给MSE乘上数据点的数量，即可得到**残差平方和(Residual sum of squares, RSS)** 。

MSE在认知建模中往往应用在对连续变量的数据的建模上，另外RSS还可以在计算AIC和BIC时替代对数似然函数。

#### 对数似然(Log-likelihood)



对数损失似然函数的公式为：

$L=\prod (P_i) = p(x_1) * p(x_2) * ...* p(x_n)$ 

似然(likelihood)表示每个数据在当前模型下发生的可能性的乘积

$LL =log(L)=\sum log(P_i)$ 

由于乘积的形式会得到一个很小的数，不便于后续处理，因此对数似然将似然取对数以便比较。


横轴为待评估的模型的各参数值，左侧纵轴为似然的值，右侧纵轴为对数似然的值


![Image Name](https://www.aptech.com/wp-content/uploads/2020/09/poisson-likelihood-function.jpeg)

资料来源：https://www.aptech.com/blog/beginners-guide-to-maximum-likelihood-estimation-in-gauss/


对数损失似然函数有一大优点，即在不论建模的数据为离散分布的选项数据还是连续分布的反应时或者评分数据时等情况都可以使用。

在计算模型的研究里，对数似然函数作为模型拟合优度的指标有两种用途。

第一，研究者可以对比不同模型的平均对数似然函数来判断模型是否对数据有很好的拟合。

第二，研究者可以使用似然比检验(Likelihood test)去判断嵌套模型之间的差异是否显著。

### 模型的泛化能力：预测准确性

前面提到的诸多模型拟合优度 (Goodness of fit) 指标，只能衡量模型对于当前样本的拟合程度。

对于样本外的数据，我们不确定该模型是否具有**泛化能力**，即该模型是否能准确的预测样本外的数据。

为了评估模型的预测能力，我们有以下3种策略：
1. 通过新数据对模型进行评估。我们可以收集新的数据，并检验模型的预测能力。
2. 从已有样本中拿出一部分数据用来预测。即交叉验证。
3. 通过统计方法进行近似。比如使用对交叉验证近似的信息熵指标来评估模型的泛化能力。

#### 交叉验证

收集新数据来检验模型的预测能力是一种理所当然的直觉。但心理学数据不同于其他学科的数据，它常受到**时间因素**和**抽样**的影响。
- 比如，心境可能随着季节变化，因此在不同季节收集到的数据会受到时间的影响。

因此，一种更高效的方法是，一次性多收集一些数据，选择其中的一部分作为预测数据。

但问题在于，我们选择哪一部分数据作为预测数据呐？或者说，我们该如何有效的对数据进行抽取呐？

**交叉验证**的目的就在于：提供不同的抽取预测数据的策略

常见的交叉验证策略：
1. 分半交叉验证 (Split-half cross-validation)
	- 分半交叉验证将观测数据对半分成两部分，分别在不同的数据集上拟合模型，并在另外一半数据集上验证模型，最后再对比不同的模型在两份数据集作为验证集时的预测准确度。
2. K 折交叉验证 (K-fold cross-validation) 
	- K 折交叉验证把数据分成 K 分，其中一份作为训练集，其余的 K-1 分数据集作为验证集，总共重复这个流程 K 次。以 K 次验证结果的均值作为验证标准。
3. 留一法交叉验证 (Leave-one-out cross-validation)
	- 留一法交叉验证是 K 折交叉验证的一个特例，当分折的数量等于数据的数量时，K 折留一法便成了留一法交叉验证。留一法交叉验证相较于普通的交叉验证方法，几乎使用了所有数据去训练模型，因此留一法交叉验证的训练模型时的**偏差 (bias) 更小、更鲁棒**，但是又因为验证集只有一个数据点，验证模型的时候**留一法交叉验证的方差 (Variance) 也会更大**。

K 折交叉验证图示
![](https://hub.packtpub.com/wp-content/uploads/2019/05/KFold.png)

source: https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/

#### 信息准则(information criteria)

尽管交叉验证存在诸多优势，并且避免了收集数据的潜在问题，但是交叉验证**在认知神经科学里的认知建模领域里的使用并不广泛**，

其最主要的原因在于，研究者收集的数据**样本量往往有限**，然而很多计算模型却对数据样本量有所需求，如果使用交叉验证，将数据拆分，那么很有可能导致拟合模型的试次数量不足，使得模型拟合和验证的结果较差，进而导致产生一类错误和二类错误的概率增大。

为了解决这些问题，统计学家们凭借数学与统计工具可以近似得到与交叉验证相似的结果。

这类近似交叉验证的指标被称为**信息准则**(information criteria)。

信息准则是对统计模型**预测精度**的一种度量。它考虑**模型与数据的匹配程度**，并通过**模型的复杂性**进行惩罚。

从前面关于模型**偏差方差权衡** (Bias variance trade-off)的讨论中我们了解到
- **偏差 bias** 会随着模型的复杂度的增大而减小，偏差大的模型往往*欠拟合*；
- **方差 error** 则会随着模型复杂度的增大而增大，方差大的模型往往*过拟合*。
- 方差和偏差之间存在一种**权衡关系**。
- 一般来说，复杂的模型容易过拟合，而简单的模型则容易欠拟合。

其中：
- **模型与数据的匹配程度**更多的体现在模型的**偏差 bias**上。
- 而**模型的复杂性**更多的体现在模型的**方差 error**上。

通过稍微数学化的语言表达就是：

$信息准则 = 偏差 + 方差 = bias + error$
- 其中 bias 往往与模型的似然值相关。可以想象，似然值越大，模型的偏差越小。
- error 与模型的复杂程度相关。比如，模型的参数越多模型越复杂。

可见，信息准则越小，偏差和方差就越小。因此，**信息准则越小，代表模型的预测性越好**。

常见的4类信息准则：
1. AIC (Akaike information criterion)
2. DIC (Deviance information criterion)
3. WAIC (Widely applicable information criterion)
4. LOO-CV (Leave-one-out cross-validation)

#### 1. AIC (Akaike information criterion)

AIC是最简单的信息准则标准，由日本统计学家赤池弘次 (Hirotugu Akaike) 提出 (Akaike, 1974)，是频率主义统计学里最为经典的模型比较指标之一。

其表达式如下：
$$
A I C= bias + error = -2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)+2 p_{A I C}
$$

别被公式吓到🧐，它的原理其实很简单。

$A I C= bias + 2 p_{A I C}$

- 其中，$2 p_{A I C}$ 就是 error，代表模型参数数量的两倍。
- error 描述了模型的复杂程度。即参数越多，模型越复杂，相应的 AIC 值越大，模型预测性越差。

$A I C = -2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)+2 p_{A I C}$

- $-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$ 为 bias，描述了模型对于当前数据的匹配程度。
- bias 与 似然函数 $p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$ 相关。$\hat{\theta}_{m l e}$为最大似然法求得的参数值。 
  
  我们知道，模型拟合的越好，似然值越大，因此$\sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$越大。相应的 $-2\sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)$ 的值越小。那么 AIC的值也越小。

需要注意的是 AIC 只考虑了最大似然对应的参数值 $\hat{\theta}_{m l e}$, 因此它适用于频率学派模型的评估。而对于贝叶斯学派来说，**由于参数为分布，因此不能使用AIC来评估模型**。

#### 2. DIC (Deviance information criterion)

为了解决 AIC 无法评估贝叶斯模型。加之 AIC 只考虑到了**模型参数数量**所带来的复杂性。

统计学家们提出了"贝叶斯参数估计版的 AIC"，即 DIC (Deviance information criterion) 。

$$
\mathrm{DIC} = -2 \sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right) +2 p_{D}
$$

可以看到，DIC 与 AIC 的公式非常相似。

- 其中$-2 \sum_{i}^{n} \log p\left(y_{i} \mid \bar{\theta}\right)$ 为 bias。也称为 deviance，即 DIC 的 D。
- 与 AIC 的区别在于，由于贝叶斯模型中的参数 $\theta$ 为概率分布，DIC 选择用参数分布的均值 $\bar{\theta}$ 来替代 AIC 中的 $\hat{\theta}_{m l e}$。


![Image Name](https://cdn.kesci.com/upload/image/rkvkoshdci.png?imageView2/0/w/640/h/640)

DIC 通过 deviance 评估模型的预测性，避免了 AIC 只考虑参数数量复杂性对于模型的影响。

DIC 对于 error 的理解与 AIC 不同。DIC 考虑了参数后验分布中所有采样对于数据的预测性。

$$
\frac{1}{2}error = p_{D} = \bar{D} - D_{\hat{\theta}}
$$

- 要理解 DIC 中的 error 与 DIC 如何评估后验分布中不同参数采样的影响，关键在于了解 D (deviance) 的意义。
- D (deviance) 的公式为 $-2 \sum_{i}^{n} \log p\left(y_{i} \mid \theta\right)$。
- 可见，bias 也可以用 D 来表示，即 $D_{\hat{\theta}}$ 。差异在于，公式中的 $\theta$ 变为 $\hat{\theta}$。 并且这个 bias $\hat{\theta}$ 也出现在 $p{D}$ 的公式中。
- 有了 $\hat{\theta}$，只需要再得到 $\bar{D}$ 就行了。我们可以用参数后验分布中所有采样(如n个采样)求得，n个 $D_i$， i为1到n。那么这 n个 $D_i$ 的均值就可以表示**参数后验分布中所有采样对于数据的平均预测性**，即 $\bar{D}$。
- $p_{D} = 所有参数采样 D 的均值 - 参数分布均值的D = \bar{D} - D_{\hat{\theta}}$
- 可以想象，当两种 D 的差异最小时，$p_{D} =0$，此时模型的预测性最好，DIC值也更低。

DIC  是心理学领域最常用的模型评估指标之一。
- 与 AIC 只是简单地使用了参数数量作为复杂度的惩罚项不同的是，DIC **利用了 MCMC 采样得到的计算模型参数的后验分布去计算模型的有效数量**。
- 另外 DIC的计算速度与AIC一样很快，这与后面会介绍的其他指标形成对比。
- 最后，除了使用参数分布的均值去计算 DIC 中 bias，也可以**使用参数分布的中位数等计算 bias，这提高了 DIC 计算的灵活性**。

#### 3. WAIC (Widely applicable information criterion)

DIC 很好的解决了 AIC 无法适用于贝叶斯模型的问题，并且考虑了参数后验分布的影响。

DIC 的问题在于：
- 如果参数后验分布不是正态分布，那么 $D_{\hat{\theta}}$ 的估计会存在偏差。
- 这进而反应为，$p{D}$ 是负数。$p{D}$ 为负数导致整个 DIC 为负数，也是 DIC 的一大问题。

为了解决上面的问题，日本统计学家渡边澄夫提出了“DIC 的升级版”的升级版，WAIC (Widely applicable information criterion)。

$$
W A I C=\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)-\sum_{i}^{n}\left({V}^{s}_{j} \log p\left(Y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)
$$

![Image Name](https://cdn.kesci.com/upload/image/rkvkzpct7z.png?imageView2/0/w/482/h/480)

$\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)$ 为 bias 与 似然函数有关。
- $\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right) = \sum_{i}^{n} \log \bar{L}$，L 为似然函数。
- $\bar{L}$ 类似于 $\bar{D}$，是参数后验分布中所有参数对应的似然值的均值。
- 区别在于：
  - WAIC 每次选择一个数据点，计算它在所有后验采样上的似然值 $\sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right) = \sum_{i}^{n} \log \bar{L}$，
  - 再求这些似然值在不同后验上的平均值 $\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right) = \sum_{i}^{n} \log \bar{L}$，
  - 最后将不同数据点上的似然值求和，即 $\sum_{i}^{n} \log \left(\frac{1}{s} \sum_{j}^{S} p\left(y_{i} \mid \boldsymbol{\theta}^{j}\right)\right) = \sum_{i}^{n} \log \bar{L}$。

$\sum_{i}^{n}\left({V}^{s}_{j} \log p\left(Y_{i} \mid \boldsymbol{\theta}^{j}\right)\right)$  为 error，与 bias 不同的是，bias 计算的是似然的均值，而 error 通过 V 计算似然值的变异。
- 与上面的流程类似，首先选择一个数据点i，求得它在不同参数j下的似然值 $\log p(Y_{i} \mid \boldsymbol{\theta}^{j})$，
- 计算该数据点在不同参数采样j下似然的变异 $({V}^{s}_{j} \log p(Y_{i} \mid \boldsymbol{\theta}^{j}))$
- 最后，将每个数据点的变异加起来 $\sum_{i}^{n}({V}^{s}_{j} \log p(Y_{i} \mid \boldsymbol{\theta}^{j}))$

#### 4. LOO-CV (Leave-one-out cross-validation)

WAIC 是非常优秀且常用的指标，其本质是模型对于未知数据的预测能力的**近似**。

另一种与 WAIC 非常类似的近似方法是前文提到的**留一交叉验证法** (Leave-one-out cross-validation, LOO-CV) 

$$
ELPD_{LOO-CV} = \sum_{\mathrm{y}} \log E_{\theta}\left(L\left(y_{i}|\theta_{-i}\right)\right)
$$

$ELPD_{LOO-CV}$ 的数学表达式非常类似，其特点为：
- $ELPD_{LOO-CV}$ (expected log pointwise predictive density) 利用了留一交叉验证的思想，用去除数据点i剩下的数据拟合模型；再回过来用该模型的参数去预测数据点i。
- 因此，ELPD 与 WAIC 的 bias 非常类似。区别在于，似然中的参数值不是通过所有数据进行拟合的，而是通过去除数据点i剩下的数据拟合得到的参数 $\theta_{-i}$。
- 此外，更巧妙的是，由于 $ELPD_{LOO-CV}$ 是直接对未来数据进行预测，因此不需要再惩罚模型，即不需要 error项了。


![Image Name](https://cdn.kesci.com/upload/image/rkvl0j94s7.png?imageView2/0/w/640/h/640)


在实际操作中，我们通过 `ArViz` 的函数可以很容易的获得 WAIC 和 $ELPD_{LOO-CV}$，我们称为 **LOO** 方法。

此外，由于 $ELPD_{LOO-CV}$ 的计算量也比较大，ArViz 会使用 Pareto Smooth Importance Sampling Leave Once Out Cross Validation (PSIS-LOO-CV) 来近似 $ELPD_{LOO-CV}$。

PSIS-LOO-CV 有两大优势：
1. 计算速度快，且结果稳健
2. 提供了丰富的模型诊断指标

### Model Averaging

前面我们讨论了如何评估单个模型的**拟合优度**与**预测精度**。

但判断模型预测性能的另一个思路是：拟合多个模型，比较不同模型的预测能力。在实践中，我们对不同的模型赋予不同的权重，并组合他们生成一个元模型 (meta-model)，进而进行元预测，以此评估不同模型权重的影响。

这种给不同模型赋予权重的方法称为 模型平均法 Model Averaging。
常见有三种计算模型权重的方式：
1. Marginal likelihood
2. BIC (Bayesian information criterion)
3. Pseudo Bayesian model averaging

#### 1. Marginal likelihood

边缘似然或者边际似然 (marginal likelihood) 是贝叶斯公式的分布部分
- 即 $p(\theta|data)=\frac{p(data|\theta)p(\theta)}{p(data)}$ 中的 $p(data) = \int_{\theta}^{} p(data|\theta)p(\theta)d\theta$ 
- 边缘似然与贝叶斯公式分子部分的似然不同，表达了模型对数据的平均拟合 (Average fit)，因此它可以作为模型选择的指标。*边缘似然越大，说明模型对样本数据解释的越好*。
- 当比较两个模型时，可以将边缘似然转化为**贝叶斯因子**(Bayes factor)。

Bayesian Model Averaging

$$
w_k = \frac{e^{-ML_{k}}}{\sum e^{-ML_{k}}}
$$

通过边缘似然可以计算 Bayesian Model Averaging。
- 假设有 k 各模型。
- k个模型的边缘似然为 $ML_{k}$。
- k个模型的权重 $w_k$ 为当前模型的边缘似然 $ML_{k}$ 比上 所有模型边缘似然之和 $\sum e^{-ML_{k}}$。


#### 2. BIC (Bayesian information criterion)

因为边缘似然计算量巨大，因此我们需要一些快速的计算方式，比如 BIC (Bayesian information criterion)。

$$
A I C=-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right)+2 k
$$

$$
B I C=-2 \sum_{i}^{n} \log p\left(y_{i} \mid \hat{\theta}_{m l e}\right) + 2 k*ln(n)
$$

BIC 与 AIC 非常类似，区别在于惩罚项 error的不同。
- BIC 中的 error 在 AIC error 的基础上乘以 ln(n)。
- 其中，k为模型参数的数量；n为数据的数量。

BIC 的特点：
- 在公式上与 AIC 高度相似，因此可以用来检验模型拟合优度，其值越小，模型拟合越好。
- BIC 的 error 往往比 AIC 的更大，即惩罚更大，因此，**BIC 通常会选择简单的模型**。
- BIC 虽然适用于贝叶斯模型，但是它没有考虑先验的影响。
- 最重要的是，BIC **是边缘似然的近似**，计算速度比 ML 更快，并且同样也可以被用来计算贝叶斯因子。

#### 3. Pseudo Bayesian model averaging

BIC 虽然能近似边缘似然， 但是其 error 只考虑了参数数量和数据数量的复杂性，也没有考虑到先验的影响。
这样计算的模型权重很可能存在偏差。

为了更高效的计算模型的权重。一种可行的方法是 Pseudo Bayesian model averaging。
- 即通过 WAIC 与 LOO 来近似边缘似然。
- 再通过 Bayesian model averaging 公式计算模型权重。


$$
w_i = \frac{e^{-\Delta_{i}}}{\sum_{j}^{k} e^{-\Delta_{j}}}
$$


上面的公式与 Bayesian model averaging 的公式 $w_k = \frac{e^{-ML_{k}}}{\sum e^{-ML_{k}}}$ 很像。
区别在于：
- 通过 WAIC 或者 LOO 在模型中的差值 $\Delta_{j}$ 替代了 边缘似然 ML。
- $\Delta$ 表示的是，第 i 或者 j个模型的 WAIC 与 最优模型的 WAIC 的差值。

PyMC3 与 Arviz 提高了很多关于 Pseudo Bayesian model averaging 计算的方法，之后的实践中，我们将注重于通过 Pseudo Bayesian model averaging 展示模型平均法的作用。

总结：

模型拟合优度的方法包括：
- 拟合优度 
- mse 
- 对数似然

模型预测进度的方法包括：
- AIC
- DIC
- WAIC
- LOO

模型平均法包括：
- Bayesian model averaging
- BIC
- Pseudo Bayesian model averaging

|          | AIC                          | DIC                              | WAIC   | LOOCV  | BIC                          |
| -------- | ---------------------------- | -------------------------------- | ------ | ------ | ---------------------------- |
| 适用框架 | 频率论                       | 贝叶斯                           | 贝叶斯 | 贝叶斯 | 贝叶斯/频率论                |
| bias     | 最大似然所对应参数下的似然值 | 参数均值对应的似然 D               | ELPD   | $ELPD_{LOO-CV}$   | 最大似然所对应参数下的似然值 |
| error    | 参数数量的两倍               | $D = \bar{D} - D_{\hat{\theta}}$                      | 似然的变异   |    | 参数数量+数据数量            |

## Example

认知控制是人的一项重要的能力，在高一致性冲突和低一致性冲突的条件下，反应时间会有不同的表现。

我们建立三种不同的模型，来解释探索不同冲突条件下反应时间的分布。

数据来源：James F Cavanagh, Thomas V Wiecki, Michael X Cohen, Christina M Figueroa, Johan Samanta, Scott J Sherman, and Michael J Frank. Subthalamic nucleus stimulation reverses mediofrontal influence over decision threshold. Nature Neuroscience, September 2011. URL: http://www.nature.com/doifinder/10.1038/nn.2925, doi:10.1038/nn.2925.

In [ ]:
import pandas as pd
import numpy as np
import arviz as az
import pymc3 as pm


WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


In [ ]:
data = pd.read_csv('/home/mw/input/data7447/cavanagh_theta_nn.csv') # 载入数据
data.head(5) # 查看前5行数据

,subj_idx,stim,rt,response,theta,dbs,conf
0,0,LL,1.21,1.0,0.656275,1,HC
1,0,WL,1.63,1.0,-0.327889,1,LC
2,0,WW,1.03,1.0,-0.480285,1,HC
3,0,WL,2.77,1.0,1.927427,1,LC
4,0,WW,1.14,0.0,-0.213236,1,HC


In [ ]:
# 调用pandas对象自带的方法，将数据按照‘conf’条件进行分组，选择每组的反应时间，展示描述性统计的内容
data.groupby('conf').rt.describe() 

,count,mean,std,min,25%,50%,75%,max
conf,,,,,,,,
HC,1972.0,1.454450,0.706737,0.409,0.92975,1.295,1.8425,4.84
LC,2016.0,1.351375,0.645456,0.402,0.87775,1.200,1.6900,4.59


In [ ]:
# 调用pandas对象自带的画图功能，选择每组的反应时间，绘制概率密度函数图
data.rt.plot.density() 

<AxesSubplot:ylabel='Density'>

<Figure size 432x288 with 1 Axes>

In [ ]:
# 调用pandas对象自带的画图功能，按照‘conf’进行分组，选择每组的反应时间，绘制概率密度函数图
data.groupby(['conf']).rt.plot.density()

conf
HC    AxesSubplot(0.125,0.125;0.775x0.755)
LC    AxesSubplot(0.125,0.125;0.775x0.755)
Name: rt, dtype: object

<Figure size 432x288 with 1 Axes>

In [ ]:
# 将‘conf’条件进行0， 1编码，HC转化为1，LC转化为0，以便在pymc3中进行计算
data.conf = data.conf.map({'HC':1,'LC':0})

#### Model1: Normal

首先，我们假定反应时是呈正态分布的，正态分布的均值是由高一致性条件和低一致性条件决定的。


![Image Name](https://docs.pymc.io/en/v3/_images/continuous-18.png)

图片来源：https://docs.pymc.io/en/v3/api/distributions/continuous.html#pymc3.distributions.continuous.Normal

In [ ]:
with pm.Model() as NormalModel:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    sigma = pm.HalfNormal('sigma', sd=1)
    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0, sd=1)
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['conf'])
    # 正态分布均值是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=data['rt'] )

In [ ]:
# 展示模型结构
pm.model_to_graphviz(NormalModel)

In [ ]:
with NormalModel:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 8 seconds.


In [ ]:
# 绘制各参数的采样情况
az.plot_trace(trace,var_names=['alpha','beta','sigma','mu'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>],
       [<AxesSubplot:title={'center':'mu'}>,
        <AxesSubplot:title={'center':'mu'}>]], dtype=object)

<Figure size 864x576 with 8 Axes>

In [ ]:
# 参数的统计值
az.summary(trace,var_names=['alpha','beta','sigma'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,1.351,0.015,1.323,1.380,0.0,0.0,2361.0,2264.0,1.0
beta,0.103,0.021,0.062,0.143,0.0,0.0,2274.0,1900.0,1.0
sigma,0.677,0.007,0.662,0.690,0.0,0.0,2507.0,2856.0,1.0


In [ ]:
with NormalModel:
     # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [ ]:
# 绘制后验预测分布
az.plot_ppc(trace)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

#### Model2: Log-normal

然后，我们假定反应时是呈log正态分布的，LogNormal分布的均值是由高一致性条件和低一致性条件决定的。


![Image Name](https://docs.pymc.io/en/v3/_images/continuous-16.png)



图片来源：https://docs.pymc.io/en/v3/api/distributions/continuous.html#pymc3.distributions.continuous.LogitNormal

In [ ]:
with pm.Model() as LogNormal:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    sigma = pm.HalfNormal('sigma', sd=1)
    alpha = pm.Normal('alpha', mu=0, sd=1)
    beta = pm.Normal('beta', mu=0, sd=1)
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['conf'])
    # 正态分布均值是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Lognormal('y_obs',mu=mu,sd=sigma,observed=data['rt'] )

In [ ]:
# 展示模型结构
pm.model_to_graphviz(LogNormal)

In [ ]:
with LogNormal:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace2 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 7 seconds.


In [ ]:
# 绘制各参数的采样情况
az.plot_trace(trace2,var_names=['alpha','beta','sigma','mu'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>],
       [<AxesSubplot:title={'center':'mu'}>,
        <AxesSubplot:title={'center':'mu'}>]], dtype=object)

<Figure size 864x576 with 8 Axes>

In [ ]:
# 参数的统计值
az.summary(trace2,var_names=['alpha','beta','sigma'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.197,0.011,0.177,0.217,0.0,0.0,2079.0,2220.0,1.0
beta,0.066,0.015,0.037,0.093,0.0,0.0,2182.0,2130.0,1.0
sigma,0.465,0.005,0.455,0.475,0.0,0.0,2409.0,2485.0,1.0


In [ ]:
with LogNormal:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace2, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [ ]:
# 绘制后验预测分布
az.plot_ppc(trace2)

<AxesSubplot:xlabel='y_obs'>

<Figure size 432x288 with 1 Axes>

#### Model3: Gamma

最后，我们假定反应时是呈gamma分布的，gamma分布的alpha是由高一致性条件和低一致性条件决定的。


![Image Name](https://docs.pymc.io/en/v3/_images/continuous-6.png)

图片来源：https://docs.pymc.io/en/v3/api/distributions/continuous.html#pymc3.distributions.continuous.Gamma


In [ ]:
with pm.Model() as Gamma:
    # 先验分布:intercept, theta, beta这三个参数是随机变量
    intercept = pm.HalfNormal('intercept', sd=1)
    theta = pm.HalfNormal('theta', sd=1)
    beta = pm.HalfNormal('beta', sd=1)   
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data['conf'])
    # 参数alpha是确定性随机变量，这个变量的值完全由右端值确定
    alpha = pm.Deterministic("alpha",  intercept+ theta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Gamma('y_obs',alpha=alpha,beta=beta,observed=data['rt'] )

In [ ]:
# 展示模型结构
pm.model_to_graphviz(Gamma)

In [ ]:
with Gamma:
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace3 = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [beta, theta, intercept]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 25 seconds.


In [ ]:
# 绘制各参数的采样情况
az.plot_trace(trace3,var_names=['beta','intercept','theta'])

array([[<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'intercept'}>,
        <AxesSubplot:title={'center':'intercept'}>],
       [<AxesSubplot:title={'center':'theta'}>,
        <AxesSubplot:title={'center':'theta'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [ ]:
# 参数的统计值
az.summary(trace3,var_names=['beta','intercept','theta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta,3.359,0.078,3.220,3.510,0.002,0.001,1384.0,1143.0,1.0
intercept,4.571,0.107,4.384,4.785,0.003,0.002,1447.0,1344.0,1.0
theta,0.287,0.065,0.151,0.403,0.002,0.001,1605.0,1239.0,1.0


In [ ]:
with Gamma:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [ ]:
# 绘制后验预测分布
az.plot_ppc(trace3)

<AxesSubplot:xlabel='y_obs'>

<Figure size 432x288 with 1 Axes>

In [ ]:
# 将三个模型的采样结果进行比较
compare_dict = {"normal": trace, "log-nomal": trace2, "gamma": trace3}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
log-nomal,0,-3520.874221,2.799883,0.000000,1.000000e+00,49.771347,0.000000,False,log
gamma,1,-3596.250176,2.747013,75.375955,2.084774e-08,49.010696,8.492641,False,log
normal,2,-4102.158193,3.664165,581.283972,0.000000e+00,58.339865,29.504465,False,log


- 第一列为索引，它列出了传递给 az.compare(.) 的模型名称。

- rank 列：按照预测精度做的排名，值从0依次到模型总数，其中0代表最高精度。

- loo 列：各模型 ELPD 值的列表，总是按照 ELPD 值从最好到最差排序。

- p_loo 列：惩罚项的值列表，可以将其粗略地视为有效参数数量的估计值（但不要太认真）。此值可能低于具有更多结构的模型（如分层模型）中的实际参数数量，或者高于那些预测能力非常弱或严重错误指定的模型的实际参数数量。

- d_loo 列：每个模型与排名第一的模型之间的 LOO 相对差。因此第一个模型始终取值为0  。

- weight 列：分配给每个模型的权重。权重可以粗略地解释为在指定数据的条件下，是（参与比较的各模型中）该模型的概率。

- se 列：ELPD 的标准误差。

- dse 列：ELPD 相对差的标准误差。 dse 与 se 不一定相同，因为 ELPD 的不确定性在模型之间可能存在相关性。排名第一的模型 dse 值始终为 0 。

- warning 列：如果为True，表示这是一个警告，LOO 的近似估计不可靠。

- loo_scale 列：估计值所用的尺度。默认为对数尺度。其他选项还包括：离差值尺度，即对数分值乘以-2，这会颠倒排序，较低的 ELPD 会更好；负对数尺度，即对数分值乘以-1，与离差值尺度一样，值越低越好。